In [1]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Unbalanced Prediction

In [2]:
data = pd.read_csv('src/predictive_maintenance.csv')
print(data.info)

<bound method DataFrame.info of         UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0         1     M14860    M                298.1                    308.6   
1         2     L47181    L                298.2                    308.7   
2         3     L47182    L                298.1                    308.5   
3         4     L47183    L                298.2                    308.6   
4         5     L47184    L                298.2                    308.7   
...     ...        ...  ...                  ...                      ...   
9995   9996     M24855    M                298.8                    308.4   
9996   9997     H39410    H                298.9                    308.4   
9997   9998     M24857    M                299.0                    308.6   
9998   9999     H39412    H                299.0                    308.7   
9999  10000     M24859    M                299.0                    308.7   

      Rotational speed [rpm]  Torque [Nm]  

In [3]:
new_labels = ['uid', 'prod_id', 'prod_type', 'air_temp', 'process_temp', 'rot_speed', 'torque', 'tool_wear', 'target', 'failure_type']

data.columns = new_labels
X = data[['torque','process_temp','tool_wear']]
y = data['target']
print(y.value_counts())

target
0    9661
1     339
Name: count, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Visualizando o shape dos dados
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(7000, 3) (3000, 3) (7000,) (3000,)


In [5]:
# Treinando modelo sem balanceamento de classes
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train.ravel())

predictions = lr.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2895
           1       0.00      0.00      0.00       105

    accuracy                           0.96      3000
   macro avg       0.48      0.50      0.49      3000
weighted avg       0.93      0.96      0.95      3000



c:\Users\guilh\Downloads\ml_tcc-20230826T003817Z-001\ml_tcc\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\guilh\Downloads\ml_tcc-20230826T003817Z-001\ml_tcc\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\guilh\Downloads\ml_tcc-20230826T003817Z-001\ml_tcc\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

# NEAR MISS

In [6]:
# Antes da redução de dimensionalidade

print(f"Antes da redução de dimensionalidade -> Falha: {sum(y_train ==1)}")
print(f"Antes da redução de dimensionalidade -> Não falha: {sum(y_train == 0)}")

# Aplicando o NearMiss
from imblearn.under_sampling import NearMiss

nr = NearMiss()

X_train_miss, y_train_miss = nr.fit_resample(X_train, y_train.ravel())

print(f"Depois da redução de dimensionalidade -> Falha: {sum(y_train_miss ==1)}")

print(f"Depois da redução de dimensionalidade -> Não falha: {sum(y_train_miss == 0)}")

Antes da redução de dimensionalidade -> Falha: 234
Antes da redução de dimensionalidade -> Não falha: 6766
Depois da redução de dimensionalidade -> Falha: 234
Depois da redução de dimensionalidade -> Não falha: 234


In [7]:
# Verificando resultados do NearMiss

lr2 = LogisticRegression()
lr2.fit(X_train_miss, y_train_miss.ravel())

predictions = lr2.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.98      0.69      0.81      2895
           1       0.06      0.52      0.10       105

    accuracy                           0.68      3000
   macro avg       0.52      0.61      0.45      3000
weighted avg       0.94      0.68      0.78      3000



# SMOTE

In [8]:
# Aplicando SMOTE para balancear os dados
# Criando superamostragem

print(f"Antes da superamostragem: Não Falha: {sum(y_train == 0)}")
print(f"Antes da superamostragem: Falha: {sum(y_train == 1)}\n")

# Aplicando Superamostragem

from imblearn.over_sampling import SMOTE

sm= SMOTE(random_state=2)

X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

print(f"Depois da superamostragem: Não Falha: {sum(y_train_sm == 0)}")	
print(f"Depois da superamostragem: Falha: {sum(y_train_sm == 1)}\n")


Antes da superamostragem: Não Falha: 6766
Antes da superamostragem: Falha: 234

Depois da superamostragem: Não Falha: 6766
Depois da superamostragem: Falha: 6766



In [9]:
# Criando regressao e verificando desempenho com SMOTE

lr1 =  LogisticRegression()

lr1.fit(X_train_sm, y_train_sm)

predictions = lr1.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.72      0.83      2895
           1       0.09      0.80      0.17       105

    accuracy                           0.72      3000
   macro avg       0.54      0.76      0.50      3000
weighted avg       0.96      0.72      0.81      3000

